# <center>Segmenting and Clustering Neighborhoods in Toronto</center>
<i><center> By: Phil Murphy </center><i>

### **Create Notebook and Import Libraries - Beginning of Part 1**


New Library - BeautifulSoup - http://beautiful-soup-4.readthedocs.io/en/latest/

In [1]:
!pip install folium

In [28]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
from geopy.geocoders import Nominatim # Converts Addresses into Lat/Long Values
from pandas.io.json import json_normalize #Transforms JSON into Pandas df
import csv
from urllib.request import urlopen

import folium #Map library

#Import K-Means From Clustering Stage
from sklearn.cluster import KMeans

#Matplotlib librariesr
import matplotlib.cm as cm
import matplotlib.colors as colors



print('Libraries Imported')

Libraries Imported


<b><center> End of Part 1 </center></b>

### Scrape Data from Website - Beginning of Part 2

From: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [35]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'lxml')

print('Data Scraped by BeautifulSoup')

Data Scraped by BeautifulSoup


### Create Dataframe Using Pandas

In [36]:
table = soup.find('table',{'class':'wikitable sortable'})
#table

table_rows = table.find_all('tr')
#table_rows

data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighborhood'])
df = df[~df['PostalCode'].isnull()]  #to filter out blank rows

print(df.head(10))

   PostalCode           Borough                                 Neighborhood
1         M1A      Not assigned                                 Not assigned
2         M2A      Not assigned                                 Not assigned
3         M3A        North York                                    Parkwoods
4         M4A        North York                             Victoria Village
5         M5A  Downtown Toronto                    Regent Park, Harbourfront
6         M6A        North York             Lawrence Manor, Lawrence Heights
7         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government
8         M8A      Not assigned                                 Not assigned
9         M9A         Etobicoke      Islington Avenue, Humber Valley Village
10        M1B       Scarborough                               Malvern, Rouge


In [37]:
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180 entries, 1 to 180
Data columns (total 3 columns):
PostalCode      180 non-null object
Borough         180 non-null object
Neighborhood    180 non-null object
dtypes: object(3)
memory usage: 5.6+ KB


(180, 3)

### Create Pandas Dataframe from Scraped Data

In [38]:
import pandas
import requests
from bs4 import BeautifulSoup
website_text = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_text,'lxml')

table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pandas.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighborhood'])
df = df[~df['PostalCode'].isnull()]  #Ignore Not Assigned rows

df.head(15)

,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M8A,Not assigned,Not assigned
9,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
10,M1B,Scarborough,"Malvern, Rouge"


### Ignore "Not Assigned" values for 'Borough'

In [39]:
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
df1 = df.reset_index()
df1.head()

,index,PostalCode,Borough,Neighborhood
0,3,M3A,North York,Parkwoods
1,4,M4A,North York,Victoria Village
2,5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,6,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [40]:
df1.info()
df1.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 4 columns):
index           103 non-null int64
PostalCode      103 non-null object
Borough         103 non-null object
Neighborhood    103 non-null object
dtypes: int64(1), object(3)
memory usage: 3.3+ KB


(103, 4)

### More than one neighborhood can exist in one postal code

We will combine neighborhoods seperated by a comma into one row

In [67]:
df2= df1.groupby('PostalCode').agg(lambda x: ','.join(x))
df2= df1.reset_index()
df2.head()

,level_0,index,PostalCode,Borough,Neighborhood
0,0,3,M3A,North York,Parkwoods
1,1,4,M4A,North York,Victoria Village
2,2,5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,3,6,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,4,7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [70]:
df2.loc[df2['Neighborhood']=="Not assigned",'Neighborhood']=df2.loc[df2['Neighborhood']=="Not assigned",'Borough']
df2.info ()
df2.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 5 columns):
level_0         103 non-null int64
index           103 non-null int64
PostalCode      103 non-null object
Borough         103 non-null object
Neighborhood    103 non-null object
dtypes: int64(2), object(3)
memory usage: 4.1+ KB


(103, 5)

### Reset Table to Provide Output

In [64]:
df3 = df2.reset_index()
df3.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [65]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
PostalCode      103 non-null object
Borough         103 non-null object
Neighborhood    103 non-null object
dtypes: object(3)
memory usage: 2.5+ KB


In [66]:
df3.shape

(103, 3)

<b><center> End of Part 2 </center></b>

### Getting Lattitude and Longitude Using CSV File - Beginning of Part 3

Recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html. Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [81]:
!wget -q -O "coordinates.csv" http://cocl.us/Geospatial_data
print('Coordinates downloaded!')
df_coords = pd.read_csv('coordinates.csv')

Coordinates downloaded!


In [82]:
df_coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [83]:
df_coords.info()
df_coords.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
Postal Code    103 non-null object
Latitude       103 non-null float64
Longitude      103 non-null float64
dtypes: float64(2), object(1)
memory usage: 2.5+ KB


(103, 3)

In [86]:
df_toronto = pd.merge(df3, df_coords, how='left', left_on = 'PostalCode', right_on = 'Postal Code')
df_toronto.drop("Postal Code", axis=1, inplace=True)
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


<b><center> End of Part 3 </center></b>

### Using GeoCoder - Beginning of Part 2

In [18]:
!pip install geocoder

print ('GeoCoder Installed Successfully!')

GeoCoder Installed Successfully!


In [19]:
!pip install geopy

print ('Geopy Installed Successfully')

Geopy Installed Successfully


In [20]:
df_geo_coor = pd.read_csv("./Geospatial_Coordinates.csv")
df_geo_coor.head()

FileNotFoundError: [Errno 2] File b'./Geospatial_Coordinates.csv' does not exist: b'./Geospatial_Coordinates.csv'